# Importing 
import the importent Libraries 

In [33]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import json
import requests
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.8.0


Read the files and do some more cleaning such as removing the stoping words and remove the missing

In [34]:
with open('list.txt','r') as st:
    list_stopWords = st.read()
list_stopWords = list_stopWords.split('\n')


def stopwords(text):
    '''
    FUNCATION
    drop the arabic stopping words values from the input
    and then return the return it
    INPUT
    text of str data type
    OUTPUT
    text of str data type
    '''
    text = ' '.join([word for word in text.split(' ') if word not in list_stopWords])
    text = text.replace("\n", "")
    text = text.replace("-", "")
    text = text.replace("--", "")
    text = text.replace("››", "")
    text = text.replace("‹‹", "")
    text = text.replace("‰", "")
    text = text.replace("••", "")
    text = text.replace("•", "")
    text = text.replace("’", "")
    text = text.replace("‘", "")
    text = text.replace("|", "")   
    text = text.replace(";", "")  
    text = text.replace("?", "")  
    text = text.replace("«", "")  
    text = text.replace("·", "")  
    text = text.replace("»", "")  
    text = text.replace("اء", "")  
    text = text.replace("ءيه", "")  
    text = text.replace("ءولا", "")  
    text = text.replace("ءري", "")  
    text = text.replace("ءذون", "")  
    text = text.replace("ءد", "")  

    return text
    
df = pd.read_csv('trainCopy.csv')
df.drop(['poet_name', 'Unnamed: 0'], axis=1, inplace = True) # drop useless columns
df.poem_text = df.poem_text.str[:2000]# select only the first 2000 charecter in the poem_text
df.dropna(inplace=True) # drop the missings 
df.poem_text = df.poem_text.apply(lambda x:stopwords(x)) #remove the stopping words
df.poem_title = df.poem_title.apply(lambda x:stopwords(x)) #remove the stopping words

df.head()

,poem_id,poem_text,poem_title,poet_id,poet_cat_أفغانستان,poet_cat_إيران,poet_cat_الأردن,poet_cat_الإمارات,poet_cat_البحرين,poet_cat_الجزائر,...,poet_cat_تونس,poet_cat_سوريا,poet_cat_شعراء العراق والشام,poet_cat_عمان,poet_cat_فلسطين,poet_cat_قطر,poet_cat_لبنان,poet_cat_ليبيا,poet_cat_مصر,poet_cat_موريتانيا
0,21,عيناك غابتا نخيل ساعه السحر شرفتان يناي عنهما ...,انشوده المطر,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65546,ازال يدي قدحي ياليل اين تفرق الشرب زلت اشربها ...,اقداح احلام,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,65561,مقلتيك ارتشفت النجوم وعانقت امالي الايبه وسابق...,هوي,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66170,تزيديه لوعه يلقاك لينسي لديك اكتءابه قربي مقلت...,تزيديه لوعه,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66171,عطرت احلامي الشذي شعرك المسترسل الاسود الجو حو...,عبير,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
test = pd.read_csv('testCopy.csv')
test.head()

,Unnamed: 0,poem_id,poem_text,poem_title,poet_cat_البحرين,poet_cat_السعودية,poet_cat_العراق,poet_cat_العصر الأندلسي,poet_cat_العصر العباسي,poet_cat_سوريا,poet_cat_لبنان,poet_cat_مصر
0,0,28111,اله يعلم حلت دمها وسفكه مستحلا بعدما حرما رايت...,الله يعلم حللت دمها,0,0,0,0,1,0,0,0
1,1,28112,وكنا نرجي عيسي محمدا لينقذنا لاعج الضر والبلوي...,وكنا نرجي بعد عيسي محمدا,0,0,0,0,1,0,0,0
2,2,28113,اشكو الي اله حماتي فما يعلم لاقيت سواه عجوز سو...,اشكو الي الله حماتي فما,0,0,0,0,1,0,0,0
3,3,28114,قالوا الموفق شيعي فقلت خلاف لناس ظهر وكيف يصبح...,قالوا الموفق شيعي فقلت,0,0,0,0,1,0,0,0
4,4,28115,ابو الفضل وابن الفضل انت وتربه فغير بديع ان يك...,الفضل وابن الفضل انت وتربه,0,0,0,0,1,0,0,0


In [26]:
print(df.shape)

(56538, 30)


Count the number of Unique words in the text poems using Counter 

In [37]:
filteredDF = df.groupby('poet_id').filter(lambda x : len(x)>20)
len(np.unique(filteredDF.poet_id))

389

In [113]:
# from sklearn.utils import shuffle
# shuffled = shuffle(filteredDF).reset_index()
# print(shuffled.shape)
# shuffled.head()

In [114]:
dfSample = filteredDF[:]
print(dfSample.shape)

(54218, 30)


In [115]:
# Using Farasa API to checks the lemmatizer model of arabic words.

url = 'https://farasa.qcri.org/webapi/lemmatization/'
api_key = "XXXXXXXXXXXXXX"


import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('omw-1.4')
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)
    return clean_tokens

X = dfSample.poem_text
y = dfSample.poet_id

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdulazizalkhonain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abdulazizalkhonain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/abdulazizalkhonain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [116]:
print(X_train.shape)
print(X_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

(36326,)
(17892,)
173    1379
117    1049
195    1018
247     745
263     719
       ... 
254      12
271      12
589      11
595      11
523      10
Name: poet_id, Length: 389, dtype: int64
173    690
117    538
195    516
247    392
263    391
      ... 
315      4
629      4
434      4
232      4
468      4
Name: poet_id, Length: 389, dtype: int64


In [ ]:
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  confusion_matrix, classification_report, accuracy_score
import seaborn as sns


In [ ]:
conf_colors = ['#193F48', '#1F505B', '#2C7080', '#388EA2','#4EA4C2', '#51ADC3', '#82ACC4', '#649EAB','#9FC3CB', '#BED6DC', '#CEE0E4', '#DCE9EC']
conf_colors.reverse()# to show the colors in the confusion_matrix from thicK to light colors

def conf_matrix(actual, predicted):
    cm = confusion_matrix(actual, predicted)
    sns.heatmap(cm, xticklabels=['predicted_negative', 'predicted_positive'], 
                yticklabels=['actual_negative', 'actual_positive'], annot=True,
                fmt='d', annot_kws={'fontsize':20}, cmap=conf_colors);

    true_neg, false_pos = cm[0]
    false_neg, true_pos = cm[1]

    accuracy = round((true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg),3)
    precision = round((true_pos) / (true_pos + false_pos),3)
    recall = round((true_pos) / (true_pos + false_neg),3)
    f1 = round(2 * (precision * recall) / (precision + recall),3)

    cm_results = [accuracy, precision, recall, f1]
    return cm_results

In [108]:
LR = LogisticRegression()
LR.fit(X_train_tfidf, y_train)

LogisticRegression()

In [109]:
y_predict_LR = LR.predict(X_test_tfidf)
acc_LR = accuracy_score(y_test, y_predict_LR)
print("\nThe Training Score: {} \n".format(LR.score(X_train_tfidf,y_train)*100))
print('The Accuracy is : {}\n\n'.format(acc_LR) )
# cm_LR = conf_matrix(y_test, y_predict_LR)


The Training Score: 92.03980099502488 

The Accuracy is : 0.7845117845117845




In [110]:
GNB = GaussianNB()
GNB.fit(X_train_tfidf.toarray(), y_train)

GaussianNB()

In [111]:
y_predict_GNB = GNB.predict(X_test_tfidf.toarray())
acc_GNB = accuracy_score(y_test, y_predict_GNB)
print("\nThe Training Score: {} \n".format(GNB.score(X_train_tfidf.toarray(),y_train)*100))
print('The Accuracy is : {}\n\n'.format(acc_GNB) )
# cm_GNB = conf_matrix(y_test, y_predict_GNB)


The Training Score: 100.0 

The Accuracy is : 0.8653198653198653




### Ensemble ML Algorthims:
starting with voting

In [112]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("TRAINIG RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")
    
estimators = []
log_reg = LogisticRegression(solver='liblinear')
estimators.append(('Logistic', log_reg))

tree = DecisionTreeClassifier()
estimators.append(('Tree', tree))

svm_clf = SVC(gamma='scale')
estimators.append(('SVM', svm_clf))

voting = VotingClassifier(estimators=estimators)
voting.fit(X_train_tfidf, y_train)

evaluate(voting, X_train_tfidf, X_test_tfidf, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[125   0   0   0]
 [  0 383   0   0]
 [  0  26   0   0]
 [  0   0   0  69]]
ACCURACY SCORE:
0.9569
CLASSIFICATION REPORT:
               2           7     9    10  accuracy   macro avg  weighted avg
precision    1.0    0.936430   0.0   1.0  0.956882    0.734108      0.916505
recall       1.0    1.000000   0.0   1.0  0.956882    0.750000      0.956882
f1-score     1.0    0.967172   0.0   1.0  0.956882    0.741793      0.936031
support    125.0  383.000000  26.0  69.0  0.956882  603.000000    603.000000
TESTING RESULTS: 
CONFUSION MATRIX:
[[  8  48   0   0]
 [  0 198   0   0]
 [  0   9   0   0]
 [  0  26   0   8]]
ACCURACY SCORE:
0.7205
CLASSIFICATION REPORT:
                   2           7    9         10  accuracy   macro avg  \
precision   1.000000    0.704626  0.0   1.000000  0.720539    0.676157   
recall      0.142857    1.000000  0.0   0.235294  0.720539    0.344538   
f1-score    0.250000    0.826722  0.0   0.380952  0.720539    0.364419   
s

/Users/abdulazizalkhonain/tensorflow-test/envir/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/abdulazizalkhonain/tensorflow-test/envir/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
